In [1]:
#imports
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
from openai import OpenAI
import ollama

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
#sset up enviornment

load_dotenv()
openai=OpenAI()


In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
system_prompt="You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt="Please give a detailed explanation to the following question: " + question

In [6]:
# messages

messages=[
    {"role":"system","content":system_prompt},
    {"role":"user","content":user_prompt}
]

In [7]:
# Get gpt-4o-mini to answer, with streaming

stream=openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)

response=""
display_handle=display(Markdown(""),display_id=True)
for chunk in stream:
    response +=chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown","")
    update_display(Markdown(response),display_id=display_handle.display_id)

Sure! Let's break down the provided code snippet step by step:

python
yield from {book.get("author") for book in books if book.get("author")}


### Explanation:

1. **Context of `yield`:**
   - The keyword `yield` is used in Python to create a generator. A generator is a special type of iterator that allows you to iterate over a sequence of values without storing them all in memory at once. When the `yield` statement is encountered, the function’s state is saved until the next value is requested.

2. **`yield from`:**
   - `yield from` is a way to delegate part of a generator's output to another generator. It allows you to yield all values from a sub-generator or iterable. In this case, it is yielding values from a set comprehension.

3. **Set Comprehension:**
   - The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. Set comprehensions create a set, which is an unordered collection of unique elements.
   - Here, `book.get("author")` is called for each `book` in the iterable `books`. The `get` method is used on dictionaries (which we assume `book` is, based on the context) to retrieve the value associated with the key `"author"`. If the key does not exist, `get` returns `None` by default.

4. **Iteration and Filtering:**
   - The comprehension iterates through each `book` in the `books` list and includes the author's name in the set **only** if that author's name exists (i.e., it’s not `None` or an empty value). This is achieved by the conditional clause `if book.get("author")`.
   - This effectively filters out any books that do not have an associated author.

5. **Resulting Set:**
   - As a result, the set comprehension produces a set of unique author names from the books, excluding any entries where the author information is missing.

### Practical Example:

Suppose you have the following list of book dictionaries:

python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C"},  # No author
    {"title": "Book D", "author": "Author 1"}  # Duplicate author
]


When executing the code snippet, the following happens:
- The generator expression will only add `"Author 1"` and `"Author 2"` to the set because `"Book C"` does not have an author.
- The resulting set will be `{"Author 1", "Author 2"}` (order may vary, as sets are unordered).
- When `yield from` is executed, each author in the set will be yielded one by one.

### Use Case:
This kind of construct is useful when you want to lazily produce a list of unique authors from a potentially large dataset without consuming memory for the entire list of authors at once. It effectively provides a way to iterate over each unique author as needed.

### Conclusion:
So, the line of code you presented efficiently extracts unique authors from a collection of books, filtering out any that do not have author information, and yields each author one at a time. This is useful in contexts such as data processing or building output streams in applications where memory conservation is important.

In [8]:
# Get Llama 3.2 to answer

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

**Code Explanation**

The provided code snippet is written in Python 3.5+ and utilizes the `yield` keyword, which allows us to implement generators.

Here's a breakdown of what this specific line does:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

This line can be read as: "yield all authors found in `books`".

Let's break it down further:

- `{... for ...}` is a dictionary comprehension (also known as a dictionary generator). It creates an iterable sequence of key-value pairs.

- `book.get("author")` retrieves the value associated with the key `"author"` from each `book` dictionary. If `"author"` is not present in the dictionary, it returns `None`.

- `for book in books if book.get("author")` filters the dictionaries to only include those that have an `"author"` key.

- `yield from {...}` takes the iterable sequence created by the dictionary comprehension and yields each of its values individually.

**Why**

This code snippet appears to be used in a context where we need to extract authors from a list of book dictionaries. The resulting iterator would yield one author at a time, which can be useful for various purposes such as:

- Processing books one-by-one while keeping track of the current author.
- Creating an iterator that can be paused and resumed later (more on this in the next section).

Here's some example code to illustrate how you might use this:

```python
def extract_authors(books):
    return yield from {book.get("author") for book in books if book.get("author")}

# Usage:
for author in extract_authors([
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "", },
    {"title": "Book 3", "author": "Author C"}
]):
    print(f"Processing book by {author}...")
```

**Generator Functions**

To fully understand the `yield from` syntax, you should know that generator functions are a type of function in Python that use the `yield` keyword to produce a series of values over time.

Here's a simple example of how you can create and use a generator function:

```python
def infinite_sequence():
    n = 0
    while True:
        yield n
        n += 1

# Usage:
seq_gen = infinite_sequence()
for _ in range(5):
    print(next(seq_gen))
```

In this example, `infinite_sequence` is a generator function that yields the numbers from 0 to infinity. We use the `next()` function to retrieve each value produced by the generator.

Keep in mind that generators only store the state of the current iteration and do not consume memory like lists or other data structures would. This makes them particularly useful for dealing with large datasets that don't fit into memory at once.

# Modify
Update such that the question is taken as input and sent to the model for response

In [9]:
def get_model_responses(question):
    """
    Takes a question as input, queries GPT-4o-mini and Llama 3.2 models, 
    and displays their responses.
    
    Args:
        question (str): The question to be processed by the models.
    """
    # system_prompt is already declared above lets generate a new user prompt so that the input question can be sent
    user_input_prompt = f"Please give a detailed explanation to the following question: {question}"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input_prompt}
    ]
     # GPT-4o-mini Response with Streaming
    print("Fetching response from GPT-4o-mini...")
    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

    response_gpt = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response_gpt += chunk.choices[0].delta.content or ''
        response_gpt = response_gpt.replace("```", "").replace("markdown", "")
        update_display(Markdown(response_gpt), display_id=display_handle.display_id)

    # Llama 3.2 Response
    print("Fetching response from Llama 3.2...")
    response_llama = ollama.chat(model=MODEL_LLAMA, messages=messages)
    reply_llama = response_llama['message']['content']
    display(Markdown(reply_llama))

In [ ]:
 # Prompt user for their question
my_question = input("Please enter your question: ")
# Fetch and display responses from models
get_model_responses(my_question)

Please enter your question:  What is Langgraph


Fetching response from GPT-4o-mini...


Langgraph is a conceptual framework and set of tools designed to enhance the development and deployment of applications that utilize large language models (LLMs). While specific implementations or libraries named "Langgraph" may vary or evolve, the general idea revolves around leveraging the capabilities of LLMs in a structured way that facilitates the development of complex applications.

### Key Features and Components of Langgraph

1. **Graph-Based Approach**:
   - **Graph Structures**: Langgraph utilizes graph structures to represent relationships between various language components, making it easier to understand, visualize, and manipulate the connections between different elements within a natural language processing (NLP) application.
   - **Nodes and Edges**: In a graph, nodes represent entities such as tasks, prompts, or data points, while edges represent the relationships or interactions between these entities. This structure can be beneficial for organizing and managing data and processes in LLM applications.

2. **Integration with Language Models**:
   - Langgraph is designed to interface with existing LLMs from providers like OpenAI, Google, or Hugging Face. It provides a streamlined way to utilize these models for various tasks, including text generation, summarization, sentiment analysis, and more.
   - Users can connect nodes in the graph to invoke different model capabilities, allowing for modular and reusable design patterns.

3. **Enhanced Workflows**:
   - By structuring workflows as graphs, developers can create complex applications that involve multiple steps or processes, each represented as a node. This modular approach allows for easy debugging, maintenance, and iteration.
   - For instance, a user can define a workflow that first processes text input to extract entities and then feeds these into another node that generates a summary based on the extracted information.

4. **Visualization and Understanding**:
   - Langgraph might include tools for visualizing the graph structure, making it easier for developers and stakeholders to understand how data flows through the application and how different components interact with each other.
   - Such visualization aids in explaining complex interactions and relationships within an application that uses LLMs.

5. **Support for Customization**:
   - Developers can extend the framework to cater to their specific needs by adding custom nodes that represent unique processing tasks or by integrating additional data sources and model types.

### Use Cases

Langgraph can be particularly useful in several applications:
- **Chatbots and Virtual Assistants**: Creating conversational agents that need to manage context, maintain state, and respond to user queries in a structured way.
- **Data Enrichment**: Automating workflows that involve data extraction, transformation, and loading (ETL) processes using natural language interfaces.
- **Content Creation**: Managing content production workflows where different models or prompts interact to generate articles, blog posts, or other content types.

### Conclusion

In summary, Langgraph is a powerful concept aimed at leveraging large language models through a graph-based approach. It promotes structured application development and allows for better organization, visualization, and manipulation of tasks related to natural language processing. By integrating various language models and representing tasks as interconnected nodes, Langgraph helps streamline the creation of sophisticated NLP applications, making it a valuable tool for developers in the AI landscape.

Fetching response from Llama 3.2...
